In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
stop_words  = set(stopwords.words('english'))

# Exploiter l’analyseur de sentiment déjà entraîné fourni par NLTK

NLTK intègre déjà un outil d’analyse de sentiment prêt à l’emploi appelé VADER (Valence Aware Dictionary and sEntiment Reasoner).

Un dictionnaire intelligent conçu pour détecter les sentiments exprimés dans un texte.

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

""" 
neg : négatif
neu : neutre
pos : positif
compound : score composé allant entre -1 à 1 (où -1 == très négatif, 1 == très positif)

0 : absence du sentiment concerné
1 : présence du sentiment concerné

par exemple, 
si neg = 1: le texte est très négatif
si neg = 0: le texte n'est pas du tout négatif
si neg > 0: le texte présente des aspects négatifs
"""

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')
df = df[:1000]
df

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
995,Great game. Just cant put it down. i have it o...,1
996,What seems like another boring old gemswap gam...,1
997,"Totally love it! Very challenging, awesome gr...",1
998,I am a fan of this game. Challenges you to thi...,1


In [21]:
import re
# import contractions
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = ' '.join(lemmatized_tokens)

    return processed_text

In [23]:
df['review_propre'] = df['reviewText'].apply(preprocess_text)
df

,reviewText,Positive,review_propre
0,This is a one of the best apps acording to a b...,1,one best apps acording bunch people agree bomb...
1,This is a pretty good version of the game for ...,1,pretty good version game free . lot different ...
2,this is a really cool game. there are a bunch ...,1,really cool game . bunch level find golden egg...
3,"This is a silly game and can be frustrating, b...",1,"silly game frustrating , lot fun definitely re..."
4,This is a terrific game on any pad. Hrs of fun...,1,terrific game pad . hr fun . grandkids love . ...
...,...,...,...
995,Great game. Just cant put it down. i have it o...,1,"great game . cant put . droid , asus tablet ki..."
996,What seems like another boring old gemswap gam...,1,seems like another boring old gemswap game def...
997,"Totally love it! Very challenging, awesome gr...",1,"totally love ! challenging , awesome graphic m..."
998,I am a fan of this game. Challenges you to thi...,1,fan game . challenge think ahead . load fun . ...


In [25]:
# initialisation de NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [29]:
print(analyzer.polarity_scores(df['review_propre'][0]), df['review_propre'][0])
print(analyzer.polarity_scores(df['reviewText'][0]), df['reviewText'][0])

{'neg': 0.146, 'neu': 0.548, 'pos': 0.306, 'compound': 0.5423} one best apps acording bunch people agree bomb egg pig tnt king pig realustic stuff
{'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'compound': 0.7717} This is a one of the best apps acording to a bunch of people and I agree it has bombs eggs pigs TNT king pigs and realustic stuff


In [31]:
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment

In [35]:
df['sentiment'] = df['review_propre'].apply(get_sentiment)
df

,reviewText,Positive,review_propre,sentiment
0,This is a one of the best apps acording to a b...,1,one best apps acording bunch people agree bomb...,1
1,This is a pretty good version of the game for ...,1,pretty good version game free . lot different ...,1
2,this is a really cool game. there are a bunch ...,1,really cool game . bunch level find golden egg...,1
3,"This is a silly game and can be frustrating, b...",1,"silly game frustrating , lot fun definitely re...",1
4,This is a terrific game on any pad. Hrs of fun...,1,terrific game pad . hr fun . grandkids love . ...,1
...,...,...,...,...
995,Great game. Just cant put it down. i have it o...,1,"great game . cant put . droid , asus tablet ki...",1
996,What seems like another boring old gemswap gam...,1,seems like another boring old gemswap game def...,1
997,"Totally love it! Very challenging, awesome gr...",1,"totally love ! challenging , awesome graphic m...",1
998,I am a fan of this game. Challenges you to thi...,1,fan game . challenge think ahead . load fun . ...,1


In [43]:
from sklearn.metrics import classification_report 
print(classification_report(df['Positive'], df['sentiment']))

              precision    recall  f1-score   support

           0       0.49      0.20      0.28       112
           1       0.91      0.97      0.94       888

    accuracy                           0.89      1000
   macro avg       0.70      0.59      0.61      1000
weighted avg       0.86      0.89      0.86      1000

